## You will be using NYC taxi ride data. There are two files located in the `data/nycTaxiData/` folder: `trip_fare_500k.csv` and `trip_data_500k.csv`.



`trip_fare_500k.csv` file found in the `data/nycTaxiData/` folder. 
This dataset contains a fairly large number of distinct trips taken in cabs in the NYC area in 2013 (500 thousand of them, to be exact!).

The dataset contains the following information at the top of the file (this is called the header):

* `medallion`: The ID of the cab being operated
* `hack_license`: The ID of the person operating the cab
* `vendor_id`: The type of vendor operating the cab, can either be `CMT` or `VTS`, no clue what these two types mean
* `pickup_datetime`: The time when the ride started
* `payment_type`: How the trip was paid, `UNK` stands for unknown, I have no idea what `NOC` stands for, but lets assume its some known way to pay
* `fare_amount`: Base fare cost of the trip
* `surcharge`: Additional charges that are not tolls
* `mta_tax`: The mta has to get its cut, right? :)
* `tip_amount`: How generous the rider(s) decided to be
* `tolls_amount`: How much money you had to pay in tolls
* `total_amount`: How much the trip cost.

Here are the columns of the trip dataset, found in `trip_data_500k.csv`:

* `medallion`: The ID of the cab being operated
* `hack_license`: The ID of the person operating the cab
* `vendor_id`: The type of vendor operating the cab, can either be `CMT` or `VTS`, no clue what these two types mean
* `rate_code`: Designates the kind of ride this is, must be `1` through `6`, any other number is incorrect
* `store_and_fwd_flag`: Can be either `Y`,`N`, or Nan
* `pickup_datetime`: The time when the ride started
* `dropoff_datetime`: The time when the ride ended
* `passenger_count`: The number of passengers during the ride
* `trip_time_in_secs`: How long the trip took
* `trip_distance`: Distance of the trip, to the nearest 1/10 mile
* `pickup_longitude`: Longitude of pickup location
* `pickup_latitude`: Latitude of pickup location
* `dropoff_longitude`: Longitude of dropoff location
* `dropoff_latitude`: Latitude of dropoff location

First step - make your own copy of the notebook.

Use the rest of this notebook to work through all these questions. 

If you can tackle all of these questions, then you've learned a lot already! 

For tips and commands, see the pandas class notebooks or https://github.com/guipsamora/pandas_exercises.

If not, don't worry, this stuff is hard and T.J. and Ramesh will gladly help/guide you through all of this. Contact us through Slack with any questions.

But take charge of your learning! This means:

* Ask a classmate
to help if you don't understand something. 
* If your neighbor can't help you, try using:
  * [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/index.html)
  * [google](http://www.google.com)
  * [stackoverflow](http://stackoverflow.com) to see if someone in the internet ether has had a similar problem before
  * if none of this works, then I will gladly help you
* This will accomplish at least two things:
  * It will get you to use online resources and take charge of your learning
  * Get you to learn alternative approaches

I've started the bare-bones script for you by:

* importing what you will need.
* loading the two datasets into `DataFrame` objects (you might need to change the path to where the file is located on your system).
* formatting the timestamp for you so that you don't have to figure out how to do it, because spending 30+ minutes  (or more) trying to figure it out is not the point of the assignment. This way, all of the functions in `fareData.pickup_datetime.dt` can immediately be used on the `pickup_datetime` column your dataset.

The rest I leave to you. Happy hacking!

In [1]:
from __future__ import print_function, unicode_literals, division
import pandas as pd
import numpy as np

## Let's start with the fare data:

In [2]:
fareData = pd.read_csv("../data/nycTaxiData/trip_fare_500k.csv")
fareData.pickup_datetime = pd.to_datetime(fareData.pickup_datetime,format="%Y-%m-%d %H:%M:%S")
fareData.dtypes #this is to confirm that the pickup_datetime column, as well as all of the other
# columns, are in the appropriate formats (pickup_datetime should be in datetime64 format)
# if it isn't something is wrong, and we need to figure what that is

medallion                  object
hack_license               object
vendor_id                  object
pickup_datetime    datetime64[ns]
payment_type               object
fare_amount               float64
surcharge                 float64
mta_tax                   float64
tip_amount                float64
tolls_amount              float64
total_amount              float64
dtype: object

<b>Is there any missing data (null-values)?

In [3]:
fareData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 11 columns):
medallion          500000 non-null object
hack_license       500000 non-null object
vendor_id          500000 non-null object
pickup_datetime    500000 non-null datetime64[ns]
payment_type       500000 non-null object
fare_amount        500000 non-null float64
surcharge          500000 non-null float64
mta_tax            500000 non-null float64
tip_amount         500000 non-null float64
tolls_amount       500000 non-null float64
total_amount       500000 non-null float64
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 42.0+ MB


<b>What was the most expensive/least expensive trip taken?</b>

In [4]:
fareData.total_amount.min()
fareData.total_amount.max()

460.5

<b>How does the overall `total_amount` paid per ride correlate with `tip_amount` per ride?</b>

In [5]:
fareData.total_amount.corr(fareData.tip_amount)

0.67087530714019938

<b>How do they correlate for only rides with cash `payment_type`?<b>

In [6]:
fareData.payment_type.unique()
filteredDF = fareData[fareData.payment_type == 'CSH']
filteredDF.total_amount.corr(fareData.tip_amount)

0.0030334289521608037

<b>Calculate the average cost of a trip in this dataset given the following conditions:</b>
  1. Across the whole dataset
  2. Across the whole dataset when the `payment_type` is known (not `UNK`)
  3. For each `payment_type`. You can totally do this 1 by 1, but try to do this in a for loop.
  4. Which `payment_type` had the highest average cost?
  5. Which `payment_type` had the largest spread in how much people paid (largest standard deviation)?
  6. Which `payment_type` had the most generous people (had the highest average tip), including unknown payment types?
  7. What hour in the day were people most generous, on average, when they got into a cab?
  8. What hour of the day did people fluctuate the most in terms of tips? That is, do some hours lead to unpredictable tip amounts? 

In [7]:
print("----------- 1: ------------")
print(fareData.total_amount.mean())

print("----------- 2: ------------")
print(fareData[fareData.payment_type != 'UNK'].total_amount.mean())

print("----------- 3: ------------")
paytypes = fareData.payment_type.unique()
for paytype in paytypes:
    print("Average total fare when paid by " + paytype)
    print(fareData[fareData.payment_type == paytype].total_amount.mean())

print("----------- 4: ------------")
print('Answered above')

print("----------- 5: ------------")
paytypes = fareData.payment_type.unique()
for paytype in paytypes:
    print("STD of fare when paid by " + paytype)
    print(fareData[fareData.payment_type == paytype].total_amount.std())

print("----------- 6: ------------")
for paytype in paytypes:
    print("Average tip amount when paid by " + paytype)
    print(fareData[fareData.payment_type == paytype].tip_amount.mean())

print("----------- 7: ------------")
fareData['hour'] = fareData.pickup_datetime.dt.hour
meanByHourDF = fareData[['hour', 'tip_amount']].groupby('hour').mean().reset_index()
print(meanByHourDF[meanByHourDF.tip_amount == meanByHourDF.tip_amount.max()])

print("----------- 8: ------------")
stdByHourDF = fareData[['hour', 'tip_amount']].groupby('hour').std().reset_index()
print(stdByHourDF[stdByHourDF.tip_amount == stdByHourDF.tip_amount.max()])

----------- 1: ------------
14.17029444
----------- 2: ------------
14.1630248047
----------- 3: ------------
Average total fare when paid by CSH
11.6167465066
Average total fare when paid by DIS
5.75
Average total fare when paid by NOC
3.0
Average total fare when paid by CRD
16.371266867
Average total fare when paid by UNK
22.1166739606
----------- 4: ------------
Answered above
----------- 5: ------------
STD of fare when paid by CSH
9.5683194441
STD of fare when paid by DIS
2.47487373415
STD of fare when paid by NOC
nan
STD of fare when paid by CRD
13.7538825481
STD of fare when paid by UNK
21.0492771964
----------- 6: ------------
Average tip amount when paid by CSH
7.43515253916e-05
Average tip amount when paid by DIS
0.0
Average tip amount when paid by NOC
0.0
Average tip amount when paid by CRD
2.42857431205
Average tip amount when paid by UNK
3.43008752735
----------- 7: ------------
   hour  tip_amount
5     5    1.942813
----------- 8: ------------
   hour  tip_amount
5     5

<b>Which person (`hack_license`) made the most money:</b>
  1. In total
  2. On a per-trip basis, given that they took at least 20 trips

In [8]:
print("----------- 1: ------------")
totalPerDriver = fareData[['hack_license', 'total_amount']].groupby('hack_license').sum()
print(totalPerDriver[totalPerDriver.total_amount == totalPerDriver.total_amount.max()])

print("----------- 2: ------------")
totalPerDriver2 = fareData[['hack_license', 'total_amount']] \
    .groupby('hack_license') \
    .agg(['sum', 'count']) \
    .reset_index() 
totalPerDriver2['dollarPerTrip'] = \
    totalPerDriver2.total_amount['sum']/totalPerDriver2.total_amount['count']
print(totalPerDriver2[totalPerDriver2.dollarPerTrip == totalPerDriver2.dollarPerTrip.max()])

----------- 1: ------------
                                  total_amount
hack_license                                  
CFCD208495D565EF66E7DFF9F98764DA       2517.28
----------- 2: ------------
                          hack_license total_amount       dollarPerTrip
                                                sum count              
3638  3E8BC9829EE46234B580C2DA5ED69C0C        87.99     1         87.99


<b>Does the number of trips a given cabbie takes (her/his experience) correlate with how well she/he is tipped? If so, in what direction</b>

In [9]:
fareData.groupby("hack_license")["tip_amount"].agg([np.size, np.mean]).corr()

,size,mean
size,1.000000,-0.291082
mean,-0.291082,1.000000


<b>Does the number of times a given cab is used correlate with how well the person driving the cab is tipped? That is, are there "lucky" cabs?</b>

In [10]:
fareData.groupby("medallion")["tip_amount"].agg([np.size, np.mean]).corr()

,size,mean
size,1.00000,-0.34822
mean,-0.34822,1.00000


<b>Which `vendor_id` had the higher average `surcharge` on a per-hour basis?</b>


In [23]:
perVendorMeans = fareData.groupby(["vendor_id","hour"])["surcharge"].mean().unstack(level=1)
perVendorMeans = perVendorMeans.mean(axis=1)
print(perVendorMeans[perVendorMeans==perVendorMeans.max()])

vendor_id
VTS    0.293235
dtype: float64


<b>Which hour in the day: </b>
  1. Did people most frequently take rides?
  2. Did people least frequently take rides?
  3. Had the largest number of unique cabs on the street?
  4. Had the least number of cabs in the street?
  5. What is the average number of cabs on the streets in NYC in each quarter of the day (at least in this dataset?)?

In [30]:
print("----------- 1: ------------")
hourGroupsSizes = fareData.groupby("hour").size()
print("The hour with the most rides is:")
print(hourGroupsSizes[hourGroupsSizes==hourGroupsSizes.max()])

print("----------- 2: ------------")
print("The hour with the fewest rides is:")
print(hourGroupsSizes[hourGroupsSizes==hourGroupsSizes.min()])

print("----------- 3: ------------")
uniqueCabsHour = fareData.drop_duplicates(["medallion","hour"])
uniqueCabsPerHour = uniqueCabsHour.groupby("hour").size()
print("The hour with the most unique cabs on the street:")
print(uniqueCabsPerHour[uniqueCabsPerHour==uniqueCabsPerHour.max()])

print("----------- 4: ------------")
print("The hour with the fewest unique cabs on the street:")
print(uniqueCabsPerHour[uniqueCabsPerHour==uniqueCabsPerHour.min()])

print("----------- 5: ------------")
fareData["quarterDay"] = pd.cut(fareData.hour,[-1,5,11,17,np.inf])
earliestDay = fareData.pickup_datetime.dt.dayofyear.min()
lastDay = fareData.pickup_datetime.dt.dayofyear.max()
totalQuarters = (lastDay-earliestDay)*4
uniqueCabsQuarterDay = fareData.drop_duplicates(["medallion","quarterDay"])
print("The number of unique cabs per quarter of the day in this dataset is:")
print(uniqueCabsQuarterDay.groupby("quarterDay").size()/totalQuarters)

----------- 1: ------------
The hour with the most rides is:
hour
12    33564
dtype: int64
----------- 2: ------------
The hour with the fewest rides is:
hour
4    3962
dtype: int64
----------- 3: ------------
The hour with the most unique cabs on the street:
hour
14    6201
dtype: int64
----------- 4: ------------
The hour with the fewest unique cabs on the street:
hour
5    1935
dtype: int64
----------- 5: ------------
The number of unique cabs per quarter of the day in this dataset is:
quarterDay
(-1, 5]      73.460526
(5, 11]      86.302632
(11, 17]     92.434211
(17, inf]    91.552632
dtype: float64


<b>Join the trip_data and fare_data datasets together. You will need to join the datasets on more than one column, but you will have to figure out what those columns are!</b>

In [31]:
tripData = pd.read_csv("../data/nycTaxiData/trip_data_500k.csv")
tripData.pickup_datetime = pd.to_datetime(tripData.pickup_datetime,format="%Y-%m-%d %H:%M:%S")

In [32]:
full_data = tripData.merge(fareData,on=["medallion","hack_license","vendor_id","pickup_datetime"])
full_data.head(1)

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,hour,quarterDay
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,...,40.751171,CSH,6.5,0.0,0.5,0.0,0.0,7.0,15,"(11, 17]"


<b>Which driver (`hack_license`) carried the most passengers, on average?</b>

In [33]:
full_data.groupby("hack_license")["passenger_count"].mean().sort_values(ascending=False).head(1)

hack_license
DF1338A98DAA39B20B528EEC54081A3D    6.0
Name: passenger_count, dtype: float64

<b>How does the number of passengers correlate with the tip amount?</b>

In [34]:
full_data['passenger_count'].corr(full_data['tip_amount'])

-0.008328931582181353